<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 15:22:07
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 2.10 K (0.02%)
Current PnL: -13.19 L (-9.19%)
CY Booked + Current PnL: -6.23 L (-4.34%)
-------------------
Total profit:  4.72 L
Total loss:  -17.91 L
-------------------
Total Booked + Current PnL: 20.75 L (17.81%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.69 L (52.35%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,91.67,113445.0,9879.0,10834.0,-0.70,9.54,9.55,20.00,32.0,0.91,0.83,24.95,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-20.73,H-LC,84.38,109616.0,5304.0,12233.0,-1.01,5.08,11.16,16.81,27.0,0.43,0.80,21.95,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,19.79,173017.0,32177.0,12319.0,-0.60,22.85,7.12,31.59,88.0,2.61,1.26,50.29,XY24,NTT,BANKS
71,SBIN,760.30,863.00,-10.56,M-LC,34.38,215392.0,13912.0,13311.0,0.74,6.91,6.18,13.51,77.0,1.05,1.57,20.37,XY25,NTT,BANKS
92,VIPIND,488.80,489.00,-213.70,H-SC,98.96,81998.0,-13318.0,13357.0,1.37,-13.97,16.29,0.04,157.0,-1.00,0.60,61.73,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,7.29,179900.0,92.0,112312.0,-0.64,0.05,62.43,62.52,173.0,0.00,1.31,34.16,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,33.33,150594.0,-319.0,60765.0,-0.10,-0.21,40.35,40.06,103.0,-0.01,1.10,23.18,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,58.33,100737.0,233.0,30402.0,0.80,0.23,30.18,30.49,97.0,0.01,0.74,32.71,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.61,NaN,NaN,42.71,18530.0,-31063.0,0.0,3.60,-62.64,0.00,-62.64,270.0,-inf,0.14,24.26,XY24,ATH,MISC
85,TRIDENT,37.35,48.00,26.47,M-SC,78.65,79583.0,-12672.0,38980.0,3.53,-13.74,48.98,28.51,233.0,-0.33,0.58,36.18,XR,NTT,TEXTILES
8,AWL,325.39,485.00,-68.62,H-MC,29.17,217676.0,-50771.0,182456.0,2.59,-18.91,83.82,49.05,98.0,-0.28,1.59,11.92,XY24,NTT,FMCG
28,DIGITIDE,188.38,237.72,NaN,NaN,46.88,50042.0,8787.0,2022.0,2.14,21.30,4.04,26.19,272.0,4.35,0.37,17.81,XY24,ATH,IT
90,VALIANTORG,512.64,838.00,-360.14,H-SC,22.92,110526.0,-22760.0,107354.0,1.92,-17.08,97.13,63.47,151.0,-0.21,0.81,85.27,XR,NTT,CHEMICALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.00,-40.45,M-SC,30.21,196407.0,24786.0,137288.0,-2.67,14.44,69.90,94.43,201.0,0.18,1.44,44.96,XR,NTT,DURABLES
65,RELAXO,902.64,1176.00,-29.80,H-SC,93.75,63488.0,-52050.0,87042.0,-2.04,-45.05,137.10,30.28,131.0,-0.60,0.46,24.03,X40N,NTT,FOOTWEAR
0,5PAISA,564.75,565.00,154.81,H-SC,27.08,137332.0,-49600.0,49687.0,-2.03,-26.53,36.18,0.04,149.0,-1.00,1.00,31.38,OX40N,NTT,FINANCE
19,CAMPUS,294.86,393.00,-27.00,M-SC,3.12,152094.0,-15681.0,71530.0,-2.03,-9.35,47.03,33.28,206.0,-0.22,1.11,17.75,XY24,NTT,FOOTWEAR
34,HAPPSTMNDS,919.80,1496.95,-4.33,M-SC,69.79,66712.0,-29867.0,90468.0,-1.84,-30.92,135.61,62.75,214.0,-0.33,0.49,16.98,AR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,45.86,M-SC,21.88,127062.0,-15417.0,16035.0,0.13,-10.82,12.62,0.44,247.0,-0.96,0.93,49.97,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,26.04,151017.0,-20124.0,20070.0,0.11,-11.76,13.29,-0.03,191.0,-1.00,1.10,8.54,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.70,H-SC,98.96,81998.0,-13318.0,13357.0,1.37,-13.97,16.29,0.04,157.0,-1.00,0.60,61.73,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148800.0,-25759.0,25787.0,-0.07,-14.76,17.33,0.02,133.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,10.42,86436.0,-16216.0,16215.0,-0.02,-15.80,18.76,-0.00,59.0,-1.00,0.63,31.39,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,37.50,265785.0,-30149.0,124068.0,-0.12,-10.19,46.68,31.73,1.0,-0.24,1.94,6.38,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.47,X-LC,77.08,293220.0,31619.0,114004.0,0.68,12.09,38.88,55.67,2.0,0.28,2.14,18.93,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.44,X-LC,56.25,267938.0,-2409.0,37029.0,-0.75,-0.89,13.82,12.81,5.0,-0.07,1.96,12.06,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.22,X-LC,66.67,268617.0,39384.0,27909.0,-0.76,17.18,10.39,29.35,10.0,1.41,1.96,27.63,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.42,X-LC,25.00,82410.0,-27762.0,116833.0,0.56,-25.20,141.77,80.85,11.0,-0.24,0.60,21.38,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3043.70,NaN,NaN,94.79,76092.0,41643.0,0.0,1.45,120.88,0.00,120.89,274.0,inf,0.56,16.36,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,46.88,50042.0,8787.0,2022.0,2.14,21.30,4.04,26.19,272.0,4.35,0.37,17.81,XY24,ATH,IT
88,UNIONBANK,123.87,163.00,-1.39,M-LC,19.79,173017.0,32177.0,12319.0,-0.60,22.85,7.12,31.59,88.0,2.61,1.26,50.29,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,55.21,232694.0,7732.0,159931.0,1.24,3.44,68.73,74.53,189.0,0.05,1.70,35.10,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,25.10,H-SC,31.25,127978.0,2681.0,56400.0,-0.99,2.14,44.07,47.15,143.0,0.05,0.94,60.11,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.76,H-SC,18.75,115497.0,13493.0,31531.0,-0.41,13.23,27.30,44.14,126.0,0.43,0.84,31.62,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,19.79,173017.0,32177.0,12319.0,-0.60,22.85,7.12,31.59,88.0,2.61,1.26,50.29,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-2.41,M-LC,89.58,167214.0,16269.0,93188.0,0.73,10.78,55.73,72.51,71.0,0.17,1.22,15.18,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3043.70,NaN,NaN,94.79,76092.0,41643.0,0.0,1.45,120.88,0.00,120.89,274.0,inf,0.56,16.36,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.45,M-SC,30.21,196407.0,24786.0,137288.0,-2.67,14.44,69.90,94.43,201.0,0.18,1.44,44.96,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,89.58,167214.0,16269.0,93188.0,0.73,10.78,55.73,72.51,71.0,0.17,1.22,15.18,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-48.08,H-SC,70.83,135346.0,12010.0,124951.0,0.60,9.74,92.32,111.05,123.0,0.10,0.99,32.61,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,25.10,H-SC,31.25,127978.0,2681.0,56400.0,-0.99,2.14,44.07,47.15,143.0,0.05,0.94,60.11,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.61,NaN,NaN,42.71,18530.0,-31063.0,0.0,3.60,-62.64,0.00,-62.64,270.0,-inf,0.14,24.26,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,59.38,64474.0,-49075.0,89129.0,-1.47,-43.22,138.24,35.27,265.0,-0.55,0.47,57.80,XR,NTT,HOTELS
32,ENRIN,1377.95,3043.70,NaN,NaN,94.79,76092.0,41643.0,0.0,1.45,120.88,0.00,120.89,274.0,inf,0.56,16.36,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-17.80,H-SC,72.40,93450.0,-4530.0,26400.0,0.76,-4.62,28.25,22.32,145.0,-0.17,0.68,35.53,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-0.98,X-LC,45.83,98970.0,1225.0,36174.0,0.67,1.25,36.55,38.26,18.0,0.03,0.72,32.15,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-15.73,L-SC,100.00,169862.0,28020.0,28554.0,-0.97,19.75,16.81,39.88,262.0,0.98,1.24,72.14,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-213.70,H-SC,98.96,81998.0,-13318.0,13357.0,1.37,-13.97,16.29,0.04,157.0,-1.00,0.60,61.73,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.94,X-SC,97.92,101433.0,470.0,94059.0,-1.63,0.47,92.73,93.63,67.0,0.00,0.74,51.08,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181010.0,16340.0,52746.0,-0.91,9.92,29.14,41.95,124.0,0.31,1.32,18.13,X40N,ATH,AC
5,ASIANPAINT,2961.56,4250.00,-14.93,H-LC,95.83,169089.0,-32297.0,119918.0,1.79,-16.04,70.92,43.51,31.0,-0.27,1.24,17.70,X40,BTT,PAINTS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.93
1,25,41.89
2,50,70.56


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.39
LC    33.95
MC    21.59
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      12.97
XY25     12.05
X40N     11.92
XR       10.01
OX40N     8.20
AR        8.15
MH        1.68
X5K       1.44
X200      1.41
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.43
M-SC    15.86
H-LC    14.12
X-LC    12.90
H-MC     9.56
X-MC     7.61
M-LC     5.89
X-SC     4.48
M-MC     4.07
L-SC     2.62
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.33,33.95
IT,12.11,-9.55,63.35
FINANCE,8.50,-12.12,52.80
BANKS,7.69,-8.08,52.17
MISC,5.83,-20.49,76.96
PAINTS,5.70,-12.08,34.17
ELECTRICAL,5.47,1.49,44.75
HEALTHCARE,4.65,-1.36,30.97
AUTO,4.31,-14.78,65.25


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2656596.0,26
XR,1087957.0,13
AR,1032003.0,10
X40,681247.0,10
X40N,531148.0,11
XY25,470819.0,8
OX40N,308133.0,11
SR,176407.0,2
MH,86802.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1988243.0,20
M-SC,1774006.0,21
H-LC,607788.0,12
X-LC,543821.0,8
H-MC,526360.0,8
X-MC,413146.0,6
M-MC,412829.0,3
X-SC,307423.0,5
M-LC,253824.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      891223.0      6
M-SC       XY24      810432.0      7
H-SC       AR        528891.0      3
M-SC       XR        339381.0      4
M-MC       XY24      300517.0      2
H-MC       XY24      297191.0      3
X-LC       X40       274246.0      3
H-SC       XR        215147.0      3
           X40N      207751.0      4
H-LC       X40N      194720.0      4
M-SC       AR        185918.0      2
           OX40N     164066.0      5
X-MC       X40       155553.0      2
H-LC       X40       152216.0      3
L-SC       XR        151207.0      2
M-LC       XY24      147325.0      2
X-SC       XR        122627.0      1
M-SC       XY25      120907.0      1
X-LC       AR        116833.0      1
M-MC       XR        112312.0      1
X-LC       XY25      104300.0      3
X-MC       XY25      102502.0      1
X-SC       SR         94059.0      1
M-LC       XR         93188.0      1
H-SC       OX40N      88831.0      3
X-MC       XY24       86351.0      1
M-SC       SR         82348.0      1
H-MC       XY25       82014.0      1
H-LC       AR         81675.0      1
           X5K        71031.0      2
M-SC       X40        70954.0      1
H-LC       X200       66881.0      1
H-MC       AR         60765.0      1
L-SC       AR         57921.0      1
H-SC       MH         56400.0      1
L-MC       XR         54095.0      1
X-MC       X40N       52525.0      1
X-SC       XY24       51716.0      1
X-LC       X40N       48442.0      1
L-LC       XY25       47785.0      1
H-LC       XY24       41265.0      1
X-SC       OX40N      39021.0      2
H-MC       MH         30402.0      1
L-SC       XY24       28554.0      1
H-MC       X40        28278.0      1
           X40N       27710.0      1
X-MC       OX40N      16215.0      1
M-LC       XY25       13311.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
